## Web scraping assignment 3
* ### Name: Rajib Dutta
* ### Email: duttarajib78@gmail.com
* ### Internship Batch: DS2402

### 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.
### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: 
"Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [ ]:
import re

import pandas as pd
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait

driver=webdriver.Edge()
driver.maximize_window()
driver.get("https://www.amazon.in/")
actions = ActionChains(driver)

WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.XPATH, "//input[@id='twotabsearchtextbox']"))).send_keys('guitar')
WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.XPATH, "//input[@id='nav-search-submit-button']"))).click()

page_count=1
brands=[]
names=[]
prices=[]
deliveries=[]
availabilities=[]
urls=[]
product_df=pd.DataFrame(columns=['brand','name','price','delivery','availability','url'])
while 1:
    if page_count>3:
        break
    parent_window=driver.current_window_handle
    items=WebDriverWait(driver, 10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, '//a[@class="a-link-normal s-no-outline"]')))
    for item in items:
        try:
            url=item.get_attribute('href')
            urls.append(url)
            driver.execute_script("arguments[0].click()", item)
        except Exception as e:
            print('Item detail not found')
            continue
        child_window=driver.window_handles[-1]
        driver.switch_to.window(child_window)
        try:
            name_xpath='//div[@id="titleSection"]/h1/span'
            name=WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.XPATH, name_xpath))).text
            names.append(name)
        except Exception as e:
            print('Title not found')
            names.append('-')
        try:
            brand_xpath='//table[@class="a-normal a-spacing-micro"]/tbody/tr[1]/td[2]/span'
            brand=WebDriverWait(driver, 10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, brand_xpath)))[0].text
            brands.append(brand)
        except Exception as e:
            print('Brand not found')
            brands.append('-')
        try:
            price_xpath='//span[@class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay"]/span[2]/span[2]'
            price=WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.XPATH, price_xpath))).text
            prices.append(price)
        except Exception as e:
            print('Price not found')
            prices.append('-')
        try:
            delivery_xpath='//div[@id="deliveryBlockMessage"]/div/div[1]/span/span'
            delivery=WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.XPATH, delivery_xpath))).text
            deliveries.append(delivery)
        except Exception as e:
            print('Delivery not found')
            deliveries.append('-')
        try:
            availability_xpath='//div[@id="availability"]/span'
            availability=WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.XPATH, availability_xpath))).text
            availabilities.append(availability)
        except Exception as e:
            print('Availability not found')
            availabilities.append('-')
        driver.close()
        driver.switch_to.window(parent_window)
    try:
        next_button_xpath='//span[@class="s-pagination-strip"]/a'
        next_button=WebDriverWait(driver, 10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, next_button_xpath)))[-1]
        driver.execute_script("arguments[0].click();", next_button)
        print('Found via primary XPATH...Next page visited')
    except Exception as e:
        print('Next button primary XPATH not found, trying the secondary XPATH...')
        next_button_xpath='//span[@class="s-pagination-strip"]/ul/li/span/a'
        try:
            next_button=WebDriverWait(driver, 10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, next_button_xpath)))[-1]
            driver.execute_script("arguments[0].click();", next_button)
            print('Found via secondary XPATH...Next page visited')
        except Exception as e:
            print('Next button not enabled')
            break
    page_count+=1
driver.quit()
product_df.brand=brands
product_df.name=names
product_df.price=prices
product_df.delivery=deliveries
product_df.availability=availabilities
product_df.url=urls
with open(file='./guitars.csv', mode='w', encoding='utf-8') as f:
    product_df.to_csv(path_or_buf=f,index=False)
    f.close()

### 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [ ]:
import os
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from  selenium.webdriver.common.action_chains import ActionChains

def search_images(url, search_bar_xpath, search_button_xpath, category):
    browser=webdriver.Edge()
    browser.maximize_window()
    actions=ActionChains(browser)
    browser.get(url)
    search_bar=WebDriverWait(browser, 10).until(expected_conditions.element_to_be_clickable((By.XPATH, search_bar_xpath)))
    search_button=WebDriverWait(browser, 10).until(expected_conditions.element_to_be_clickable((By.XPATH, search_button_xpath)))
    actions.click(on_element=search_bar)
    actions.send_keys(category)
    actions.click(on_element=search_button)
    actions.perform()
    actions.reset_actions()
    return browser

search_bar_xpath='/html[1]/body[1]/div[1]/div[3]/form[1]/div[1]/div[1]/div[1]/div[1]/div[2]/textarea[1]'
search_button_xpath="/html[1]/body[1]/div[1]/div[3]/form[1]/div[1]/div[1]/div[1]/button[1]/div[1]/span[1]/*[name()='svg'][1]"
categories=['fruits','cars', 'Machine Learning', 'Guitar', 'Cakes']
category_index=0
while True:
    if category_index==len(categories):
        break
    category=categories[category_index]
    browser=search_images('https://images.google.com/', search_bar_xpath, search_button_xpath, category)    
    images=None
    try:
        images=WebDriverWait(browser, 10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, '//img[@class="YQ4gaf"]')))
    except Exception as e:
        print('Images not found...retrying')
        browser.quit()
        continue
    file_path=(f'./pictures/{category}')
    os.makedirs(file_path, exist_ok=True)
    for i, image in enumerate(images):
        if i==10:
            break
        file_name=f'{file_path}/{category}_{i}.png'
        with open(file=file_name, mode='wb') as f:
            try:
                f.write(image.screenshot_as_png)
            except Exception as e:
                print('Failed to save image...retrying')
                browser.quit()
                continue
            f.close()
    browser.quit()
    category_index+=1

### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped:
* “Brand Name”, 
* “Smartphone name”, 
* “Colour”, 
* “RAM”, 
* “Storage(ROM)”, 
* “Primary Camera”, 
* “Secondary Camera”, 
* “Display Size”, 
* “Battery Capacity”, 
* “Price”, 
* “Product URL”. 
#### Incase if any of the details is missing then replace it by “-“. Save your results in a dataframe and CSV.

In [ ]:
from selenium.common import NoSuchElementException
import pandas as pd
import re
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait

def search_phones_on_flipcart(phone_keyword):
    browser=webdriver.Edge()
    browser.maximize_window()
    browser.get('''https://www.flipkart.com/''')
    search_bar_xpath='/html[1]/body[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/header[1]/div[1]/div[2]/form[1]/div[1]/div[1]/input[1]'
    search_button_xpath="/html[1]/body[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/header[1]/div[1]/div[2]/form[1]/div[1]/button[1]/*[name()='svg'][1]"
    WebDriverWait(browser, 10).until(expected_conditions.presence_of_element_located((By.XPATH, search_bar_xpath))).send_keys(phone_keyword)
    WebDriverWait(browser, 10).until(expected_conditions.presence_of_element_located((By.XPATH, search_button_xpath))).click()
    return browser

browser=None
phones=None
while True:
    if phones!=None:
        break
    browser=search_phones_on_flipcart('Oneplus Nord')
    try:
        phones=WebDriverWait(browser, 10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, '//a[@class="CGtC98"]')))
    except Exception as e:
        print('Phones not found...retrying')
        browser.quit()
        continue
    
parent_window=browser.current_window_handle
phone_df=pd.DataFrame(columns=['Brand Name', 'Smartphone name', 'Colour', 'RAM', 'Storage(ROM)', 'Primary Camera', 'Secondary Camera', 'Display Size', 'Battery Capacity', 'Price', 'Product URL'])
for i, phone in enumerate(phones):
    url=phone.get_attribute('href')
    browser.execute_script("arguments[0].click()", phone)
    
    browser.switch_to.window(browser.window_handles[-1])
    read_more_xpath='//button[@class="QqFHMw _4FgsLt"]'
    read_more_button=browser.find_element(By.XPATH, read_more_xpath)
    browser.execute_script("arguments[0].click()", read_more_button)
    
    try:
        brand=WebDriverWait(browser,10).until(expected_conditions.presence_of_element_located((By.XPATH, '//span[@class="VU-ZEz"]'))).text
        brand=re.compile(pattern='^\w+').findall(string=brand)[0]
    except Exception as e:
        print('Brand not found',e)
        brand='-'
    try:
        phone_name=WebDriverWait(browser,10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[3]/td[2]/ul/li'))).text
    except Exception as e:
        print('Phone name not found', e)
        phone_name='-'
    try:
        color=WebDriverWait(browser,10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[4]/td[2]/ul/li'))).text
    except Exception as e:
        print('Color not found', e)
        color='-'
    try:
        ram=WebDriverWait(browser, 10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]/ul/li'))).text
    except Exception as e:
        print('RAM not found', e)
        ram='-'
    try:
        storage=WebDriverWait(browser,10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]/ul/li'))).text
    except Exception as e:
        print('Storage not found', e)
        storage='-'
    try:
        primary_camera=WebDriverWait(browser,10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[1]/td[2]/ul/li'))).text
        if len(re.compile(pattern='^\d+').findall(string=primary_camera))==0:
            primary_camera=WebDriverWait(browser,10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]/ul/li'))).text
    except Exception as e:
        print(f'{i}: Primary Camera not found', e)
        primary_camera='-'
    try:
        secondary_camera=WebDriverWait(browser,10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]/ul/li'))).text
        if len(re.compile(pattern='^\d+').findall(string=secondary_camera))==0:
            secondary_camera='-'
        elif len(re.compile(pattern='.*Rear.*').findall(string=secondary_camera))>0:
            secondary_camera='-'
    except Exception as e:
        print(f'{i}: Secondary Camera not found', e)
        secondary_camera='-'
    try:
        display_size=WebDriverWait(browser, 10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]/ul/li'))).text
    except Exception as e:
        print('Display size not found', e)
        display_size='-'
    try:
        battery_capacity=WebDriverWait(browser,10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="_1UhVsV"]/div[7]/table/tbody/tr[1]/td[2]/ul/li'))).text
        if len(re.compile(pattern='.*mAh.*').findall(battery_capacity))==0:
            raise NoSuchElementException('Battery capacity not found..retrying')
    except Exception as e:
        try:
            battery_capacity=WebDriverWait(browser,10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="_1UhVsV"]/div[9]/table/tbody/tr[1]/td[2]/ul/li'))).text
            if len(re.compile(pattern='.*mAh.*').findall(battery_capacity))==0:
                raise NoSuchElementException('Battery capacity not found..retrying')
        except Exception as e:
            print('Retry failed..No battery capacity found', e)
            battery_capacity='-'
    try:
        price=WebDriverWait(browser, 10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="_30jeq3 _16Jk6d"]'))).text.replace('₹', '').replace(',','')
    except Exception as e:
        print('Price not found')
        price='-'
    phone_df.loc[i]=(brand, phone_name, color, ram, storage, primary_camera, secondary_camera, display_size, battery_capacity, price, url)
    browser.close()
    browser.switch_to.window(parent_window)
browser.quit()
phone_df

### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [ ]:
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait

browser = webdriver.Chrome()
browser.maximize_window()
browser.get('https://www.google.com/maps')
city=input('Enter city name: ')
WebDriverWait(browser,10).until(expected_conditions.presence_of_element_located((By.XPATH, '//input[@id="searchboxinput"]'))).send_keys(city)
WebDriverWait(browser,10).until(expected_conditions.element_to_be_clickable((By.XPATH, '//span[@class="google-symbols"]'))).click()
ActionChains(browser).pause(5).perform()
try:
    result=WebDriverWait(browser,10).until(expected_conditions.presence_of_element_located((By.XPATH,'//h1[@class="DUwDvf lfPIob"]'))).text.strip()
    city_url=browser.current_url
    lat, long=city_url.split(sep='@', maxsplit=2)[-1].split(sep=',',maxsplit=3)[:-1][0], city_url.split(sep='@', maxsplit=2)[-1].split(sep=',',maxsplit=3)[:-1][-1]
    print(f'Latitude and Longitude of {result} : {lat} and {long}')
except Exception as e:
    print('City not found')
finally:
    browser.quit()

### 6. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [ ]:
import pandas as pd
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from collections import ChainMap

browser=webdriver.Chrome()
browser.maximize_window()
browser.get('https://www.digit.in/')

WebDriverWait(browser, 10).until(expected_conditions.presence_of_element_located((By.XPATH, "//input[@id='woocommerce-product-search-field-0']"))).send_keys('Gaming Laptop')
WebDriverWait(browser, 10).until(expected_conditions.presence_of_element_located((By.XPATH, "//div[@class='search head_search position-relative']//i[@class='rhicon rhi-search']"))).click()
parent_window=browser.current_window_handle

laptop_details=[]
while True:
    i=0
    while True:
        laptops=WebDriverWait(browser,10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, '//a[@class="img-centered-flex rh-flex-justify-center rh-flex-center-align"]')))
        if i==len(laptops):
            break
        laptop_link=(WebDriverWait(browser,10)
                     .until(expected_conditions.presence_of_all_elements_located((By.XPATH, '//a[@class="img-centered-flex rh-flex-justify-center rh-flex-center-align"]')))[i]
                     .get_attribute('href'))
        browser.execute_script("window.open()")
        browser.switch_to.window(browser.window_handles[-1])
        browser.get(laptop_link)
        try:
            keys=WebDriverWait(browser,10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, '//th[@class="woocommerce-product-attributes-item__label"]')))
            values=WebDriverWait(browser,10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, '//td[@class="woocommerce-product-attributes-item__value"]/p')))
        except Exception as e:
            print('Laptop details not found', e)
            browser.close()
            browser.switch_to.window(parent_window)
            i+=1
            continue
        laptop_dict={}
        for key,value in zip(keys,values):
            laptop_dict[key.text]=value.text
        laptop_details.append(laptop_dict)
        browser.close()
        browser.switch_to.window(parent_window)
        i+=1
    try:
        browser.refresh()
        next_button=WebDriverWait(browser,10).until(expected_conditions.element_to_be_clickable((By.XPATH, '//a[@class="next page-numbers"]')))
        next_button.click()
        print('Next page visited')
    except Exception as e:
        print('All available pages have been explored',e)
        break
browser.quit()
for k in ChainMap(*laptop_details):
    for laptop_dict in laptop_details:
        laptop_dict.setdefault(k,'-')
laptop_df=pd.DataFrame(laptop_details)
laptop_df

### 7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [ ]:
import re
import pandas as pd
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait

browser=webdriver.Chrome()
browser.maximize_window()
browser.get('https://www.forbes.com/billionaires/')

first_row_XPATH='/html[1]/body[1]/div[1]/div[2]/div[1]/div[1]/div[3]/div[2]/div[2]/div[1]/div[2]/div[1]/div[2]/div[1]'
rank_XPATH='//div[@class="Table_tableRow__lF_cY"]/div[1]/div'
name_XPATH='//div[@class="Table_tableRow__lF_cY"]/div[2]'
net_worth_XPATH='//div[@class="Table_tableRow__lF_cY"]/div[3]'
age_XPATH='//div[@class="Table_tableRow__lF_cY"]/div[4]'
citizenship_XPATH='//div[@class="Table_tableRow__lF_cY"]/div[5]'
source_XPATH='//div[@class="Table_tableRow__lF_cY"]/div[6]'
industry_XPATH='//div[@class="Table_tableRow__lF_cY"]/div[7]'

billionaires_df=pd.DataFrame(columns=['rank','name','net_worth','age','citizenship','source','industry'])
page_index=1
df_index=0
while True:
    first_row=WebDriverWait(browser,10).until(expected_conditions.element_to_be_clickable((By.XPATH, first_row_XPATH)))
    ActionChains(browser).move_to_element(to_element=first_row).click(on_element=first_row).perform()
    i=0
    while True:
        if i==len(WebDriverWait(browser,10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, rank_XPATH)))):
            break
            
        rank=WebDriverWait(browser,10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, rank_XPATH)))[i].text.strip()
        rank=int(re.compile(pattern='\d+').search(string=rank).group(0))
        name=WebDriverWait(browser,10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, name_XPATH)))[i].text.strip()
        new_worth=WebDriverWait(browser,10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, net_worth_XPATH)))[i].text.strip()
        age=WebDriverWait(browser,10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, age_XPATH)))[i].text.strip()
        citizenship=WebDriverWait(browser,10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, citizenship_XPATH)))[i].text.strip()
        source=WebDriverWait(browser,10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, source_XPATH)))[i].text.strip()
        industry=WebDriverWait(browser,10).until(expected_conditions.presence_of_all_elements_located((By.XPATH, industry_XPATH)))[i].text.strip()
        billionaires_df.loc[df_index]=(rank, name, new_worth, age, citizenship, source, industry)
        i+=1
        df_index+=1
    try:
        next_button=WebDriverWait(browser,10).until(expected_conditions.element_to_be_clickable((By.XPATH, '//div[@class="qXW-dtvf"]/button[7]')))
        ActionChains(browser).move_to_element(to_element=next_button).click(on_element=next_button).perform()
        page_index+=1
        print(f'Page {page_index} visited')
    except Exception as e:
        print('All available pages have been explored',e)
        break
browser.quit()
billionaires_df

### 9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [ ]:
import re
import pandas as pd
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys

browser=webdriver.Chrome()
browser.maximize_window()
browser.get('https://www.hostelworld.com/')

location_search_box_XPATH="/html[1]/body[1]/div[3]/div[1]/div[2]/main[1]/header[1]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/input[1]"
city_XPATH="//div[@class='label'][normalize-space()='London']"
lets_go_XPATH="//button[@class='large-button btn-content']//span[@class='text']"

location_search_box=WebDriverWait(browser,10).until(expected_conditions.visibility_of_element_located((By.XPATH, location_search_box_XPATH)))
ActionChains(browser).click(location_search_box).send_keys('London').perform()
location=WebDriverWait(browser,10).until(expected_conditions.visibility_of_element_located((By.XPATH, city_XPATH)))
lets_go_button=WebDriverWait(browser,10).until(expected_conditions.visibility_of_element_located((By.XPATH, lets_go_XPATH)))
ActionChains(browser).move_to_element(to_element=location).click(on_element=location).click(on_element=lets_go_button).pause(5).perform()

parent_window=browser.current_window_handle

properties_XPATH='//div[@class="property-list"]/div/div/div[5]/div'

properties=WebDriverWait(browser,10).until(expected_conditions.visibility_of_all_elements_located((By.XPATH, properties_XPATH)))
hostel_df=pd.DataFrame(columns=['name','rating','distance','total_reviews','overall_review','privates_from','dorms_from','description'])
for i,_ in enumerate(properties):
    nameWE=WebDriverWait(browser,10).until(expected_conditions.visibility_of_all_elements_located((By.XPATH, properties_XPATH)))[i]
    name=nameWE.find_element(By.XPATH, './/div[@class="property-name"]/span').text.strip()
    rating=nameWE.find_element(By.XPATH, './/span[@class="number"]').text.strip()
    distance=nameWE.find_element(By.XPATH, './/span[@class="distance-description"]').text.strip()
    total_reviews=nameWE.find_element(By.XPATH, './/span[@class="left-margin"]').text.strip()
    total_reviews=re.compile(pattern='\d+').search(string=total_reviews).group(0)
    overall_review=nameWE.find_element(By.XPATH, './/span[@class="keyword"]').text.strip()
    prices=nameWE.find_elements(By.XPATH, './/strong[@class="current"]')
    if len(prices)==2:
        privates_from=prices[0].text.strip()
        dorms_from=prices[-1].text.strip()
    else:
        privates_from='-'
        dorms_from=prices[-1].text.strip()
    hostel_detail_page_anchor=nameWE.find_element(By.XPATH,'.//a[@class="property-card-container horizontal"]')
    ActionChains(browser).key_down(Keys.CONTROL).click(on_element=hostel_detail_page_anchor).key_up(Keys.CONTROL).perform()
    browser.switch_to.window(browser.window_handles[-1])
    try:
        show_more_XPATH='/html[1]/body[1]/div[3]/div[1]/div[1]/div[2]/section[1]/div[10]/div[1]/div[2]/div[6]/div[1]/a[1]'
        description_XPATH='/html[1]/body[1]/div[3]/div[1]/div[1]/div[2]/section[1]/div[10]/div[1]/div[2]/div[6]/div[1]/div[2]'
        try:
            WebDriverWait(browser,10).until(expected_conditions.element_to_be_clickable((By.XPATH, show_more_XPATH))).click()
        except Exception as e:
            pass
        try:
            description=WebDriverWait(browser,10).until(expected_conditions.visibility_of_element_located((By.XPATH, description_XPATH))).text.strip()
        except Exception as e:
            description='-'
    except Exception as e:
        show_more_XPATH='/html[1]/body[1]/div[3]/div[1]/div[1]/div[2]/section[1]/div[10]/div[1]/div[2]/div[5]/div[1]/a[1]'
        description_XPATH='/html[1]/body[1]/div[3]/div[1]/div[1]/div[2]/section[1]/div[10]/div[1]/div[2]/div[5]/div[1]/div[2]'
        try:
            WebDriverWait(browser,10).until(expected_conditions.element_to_be_clickable((By.XPATH, show_more_XPATH))).click()
        except Exception as e:
            pass
        try:
            description=WebDriverWait(browser,10).until(expected_conditions.visibility_of_element_located((By.XPATH, description_XPATH))).text.strip()
        except Exception as e:
            description='-'
    browser.close()
    browser.switch_to.window(parent_window)
    hostel_df.loc[i]=(name,rating,distance,total_reviews,overall_review,privates_from,dorms_from,description)
browser.quit()
hostel_df